In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("train.csv")
df.head() #İlk 5 veriyi görelim.
#Verimizi tanıyarak keşfe başlayalım. 
#PassengerId :  verimizdeki yolcuları unique'leştiren, 1'den başlayıp 891'e kadar gidecek olan sıralı özellik.
#Survived : Bir yolcunun kaza sonrasında hayatta kalıp kalmadığını belirten (1 ve 0) özellik.
#Pclass : Yolcunun aldığı biletin sınıfını belirten (1: 1st class, 2: 2nd class, 3: 3rd class) özellik.
#Sex: Yolcunun cinsiyetini belirten özellik.
#Age: Yolcunun yaşını belirten özellik.
#SibSp : Kişinin yolculuk yapan çocuk sayısı
#Parch : Kişinin yolculuk yapan ebeveyn sayısı
#Ticket: Kişinin bilet numarası.
#Fare: Bilet Ücreti
#Cabin: Kişinin bulunduğu kabinin numarası.
#Embarked: Kişinin gemiye bindiği liman. (C = Cherbourg, Q = Queenstown, S = Southampton)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
test_data = pd.read_csv("test.csv")
test_data.head()
#Modelimizi oluşuturduktan sonra test edeceğimiz veriyi de yükleyelim.

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
#Exploring Patterns
#'Cinsiyet' özelliği, kişilerin kurtulmasında fark yaratan, belirleyici bir özellik mi? Bunu keşfedelim.

women = df.loc[df.Sex == 'female']["Survived"] #Burada cinsiyeti kadin olanlarin survived ozelligini cekiyoruz ve women degiskenine atiyoruz.
rate_women = sum(women)/len(women) #Sum(women): Kişi 'survive' olduysa yani kurtulduysa 1, kurtulmadıysa 0. Bunun toplamını bulmak, toplamda kurtulan kişi sayısını bulmak demektir.
#len(women) ise toplam kadın sayısını verir.

print("% of women who survived:", rate_women)

men = df.loc[df.Sex == 'male']["Survived"] #Kadınlar için yaptığımızın aynısını erkekler için yapalım.
rate_men = sum(men)/len(men)
print("% of women who survived:", rate_men)

% of women who survived: 0.7420382165605095
% of women who survived: 0.18890814558058924


Kadınlar %75 oranında kurtulurken erkekler ise yalnızca %18 oranında kurtulmuş. Cinsiyetin belirleyici bir özellik olduğunu söyleyebiliriz.

In [6]:
ages = df['Age']
maxAge = ages.max()
minAge = ages.min()
print(f"Max age: {maxAge}   Min age: {minAge}")

Max age: 80.0   Min age: 0.42


Maximum yaş 80, minimum yaş ise 0. Yaş aralığımızı 10 eşit parçaya bölerek yaşın kurtulma üzerinde etkisi olup olmadığına bakalım.

In [22]:
bins = np.linspace(0, 80, 6)  # 0 ile 80 arasında 5 eşit aralık oluşturur.

labels = [f"{int(bins[i])}-{int(bins[i+1])}" for i in range(len(bins)-1)] #Yaş aralıklarına uygun etiketler oluşturuldu.

df["Age Group"] = pd.cut(df["Age"], bins=bins, labels=labels)

In [23]:
survival_rate = df.groupby("Age Group")["Survived"].mean()
print(survival_rate)

Age Group
0-16     0.550000
16-32    0.369942
32-48    0.404255
48-64    0.434783
64-80    0.090909
Name: Survived, dtype: float64


C:\Users\Eren\AppData\Local\Temp\ipykernel_9904\1038025897.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  survival_rate = df.groupby("Age Group")["Survived"].mean()


Yaş aralıklarının örneklem sayısına bakarak bunun yanıltıcı bir veri olup olmadığını araştıralım.

In [24]:
df["Age Group"].value_counts()

Age Group
16-32    346
32-48    188
0-16     100
48-64     69
64-80     11
Name: count, dtype: int64

Özellikle 64 yaşın altındaki kişilerde yaş aralığı belirleyici bir özellik olarak kullanılabilir. ✅ 
Ki-kare testi ile bir kontrol daha yapalım.

In [21]:
from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(df["Age Group"], df["Survived"])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print(f"p-değeri: {p}")

p-değeri: 0.06756345501651245


p = 0.00194 çıktığı için, yaş gruplarının kurtulma oranı üzerinde anlamlı bir etkisi var! 🎉

Kişinin sahip olduğu Kardeş ve Eş sayısının(SibSp) kurtulması üzerindeki etkisine bakalım.

In [11]:
df.groupby("SibSp")["Survived"].mean()

SibSp
0    0.345395
1    0.535885
2    0.464286
3    0.250000
4    0.166667
5    0.000000
8    0.000000
Name: Survived, dtype: float64

In [12]:

contingency_table = pd.crosstab(df["SibSp"], df["Survived"])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print(f"p-değeri: {p}")

p-değeri: 1.5585810465902147e-06


SibSp değeri kendi başına ayırt edici bir özellik değil. ❌

In [13]:
min_fare = df["Fare"].min()
max_fare = df["Fare"].max()
print(f"Minimum Ücret: {min_fare}, Maksimum Ücret: {max_fare}")

bins = np.linspace(min_fare, max_fare, 4)  # 3 eşit aralık
df["Fare Group"] = pd.cut(df["Fare"], bins=bins, labels=[f"{bins[i]:.2f}-{bins[i+1]:.2f}" for i in range(len(bins)-1)])

Minimum Ücret: 0.0, Maksimum Ücret: 512.3292


In [14]:
fare_survival = df.groupby("Fare Group")["Survived"].mean()
print(fare_survival)

Fare Group
0.00-170.78      0.382009
170.78-341.55    0.647059
341.55-512.33    1.000000
Name: Survived, dtype: float64


C:\Users\Eren\AppData\Local\Temp\ipykernel_9904\239071266.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fare_survival = df.groupby("Fare Group")["Survived"].mean()


In [15]:
contingency_table = pd.crosstab(df["Fare Group"], df["Survived"])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print(f"p-değeri: {p}")

print(f"Gruplardaki kisi sayilari: \n{df["Fare Group"].value_counts()}")


p-değeri: 0.008032350298711032
Gruplardaki kisi sayilari: 
Fare Group
0.00-170.78      856
170.78-341.55     17
341.55-512.33      3
Name: count, dtype: int64


Ancak 3 eşit parçaya böldüğümüzde ki kare değeri bizim için anlamlı bir değere ulaştı ancak burada da aşırı dengesiz bir dağılım oldu. Fare Group kullanılabilir bir özellik değil.

In [16]:
df["Fare Group"].value_counts()

Fare Group
0.00-170.78      856
170.78-341.55     17
341.55-512.33      3
Name: count, dtype: int64

Ki-kare değeri anlamlı bir değere ulaşsa da gruplara düşen örneklem sayıları çok dengesiz oldu.